# merge_sale_dates.ipynb 

Since our model may just be guessing days after a sale has already started, we will only pick the first day of a sale and set all other subsequent days to no sale for that game

I will go through each appId and for each game, go through every day in order and find a sale. Keep that sale but for every consecutive day the sale is active, mark it as not on sale and then continue until a new sale is found.

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

full_df = pd.read_csv("all_data.csv")
full_df = full_df.sort_values(by=['appid','Date'])

for index, row in full_df.iterrows():
    if row['Initialprice'] != row['Finalprice']:
        full_df.loc[index, 'onSale'] = 1
        full_df.loc[index, 'Discount'] = round(1 - (row['Finalprice']/row['Initialprice']),3)

full_df.to_csv('all_data.csv',index=False)

This code will keep track of when a sale starts and update any days _after_ it starts to not include a sale. Essentially we are just taking the day the sale starts. We also want to make sure that we ensure its the same game since one game could have data that ends with a sale and the next one in the df could start with a sale, we do not want that to be considered the same sale.

In [17]:
def checkForSale(row, foundSale, full_df, index):
    if row['onSale'] == 1 and foundSale == False:
        foundSale = True
    elif row['onSale'] == 1 and foundSale == True:
        full_df.loc[index, 'onSale'] = 0
        full_df.loc[index, 'Discount'] = 0
        full_df.loc[index, 'Finalprice'] = row['Initialprice']
    elif row['onSale'] == 0:
        foundSale = False

    return foundSale, full_df

foundSale = False
previousId = 10

for index, row in full_df.iterrows():
    if row['appid'] == previousId:
        foundSale, full_df = checkForSale(row, foundSale, full_df, index)
    else:
        previousId = row['appid']
        foundSale = False
        foundSale, full_df = checkForSale(row, foundSale, full_df, index)

In [ ]:
full_df.to_csv('all_data_sale_start_only.csv',index=False)